In [23]:
import cv2

# Inicjalizacja kamery
video = cv2.VideoCapture(0)

# Tworzymy model tła
back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=10, detectShadows=True)

while True:
    ret, frame = video.read()
    if not ret:
        break

    # Obróć obraz (opcjonalnie)
    frame = cv2.flip(frame, 1)

    # Wydziel tło
    fg_mask = back_sub.apply(frame)

    # Przefiltruj obraz (usuń szumy)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

    # Znajdź kontury ruchu
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) < 1:
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Wyświetl wynik
    cv2.imshow("Ruch", frame)
    cv2.imshow("Maska ruchu", fg_mask)

    key = cv2.waitKey(30)
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
